# Capstone Project - Toronto

## Libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import requests

### Data Loading

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
source = requests.get(url).text
soup = BeautifulSoup(source)

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ').split(',')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
    


In [3]:
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,[Parkwoods]
1,M4A,North York,[Victoria Village]
2,M5A,Downtown Toronto,"[Regent Park, Harbourfront]"
3,M6A,North York,"[Lawrence Manor, Lawrence Heights]"
4,M7A,Queen's Park,[Ontario Provincial Government]


In [4]:
df.shape

(103, 3)

In [5]:
liste = []

for i in (df[['Borough','Neighborhood']].values).tolist():
    liste.append([i[0],i[1:]])

In [6]:
liste2 = []
for i in liste:
    for j in i[1]:
        for k in j:
            liste2.append([i[0],k])


In [7]:
d1 = pd.DataFrame(liste2)

In [8]:
df['Borough'] = d1[0]
df['Neighborhood'] = d1[1]
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,Downtown Toronto,Harbourfront
4,M7A,North York,Lawrence Manor


In [9]:
df.shape

(103, 3)

In [10]:
df2 = df[df['Borough'] != 'Not assigned'].reset_index(drop = True)
df3 = df[df['Neighborhood'] == 'Not assigned']

In [11]:
df2.shape,df3.shape

((103, 3), (0, 3))

In [ ]:
# Part 2
'-----------------------------------------------------------------------------'

### Latitude and Longitude Add

In [168]:
import sys
!conda install --yes --prefix {sys.prefix} geocoder


Solving environment: failed with initial frozen solve. Retrying with flexible solve.
Solving environment: failed with initial frozen solve. Retrying with flexible solve.

PackagesNotFoundError: The following packages are not available from current channels:

  - geocoder

Current channels:

  - https://repo.anaconda.com/pkgs/main/linux-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/linux-64
  - https://repo.anaconda.com/pkgs/r/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [18]:
data = pd.read_csv('/home/jaser/Geospatial_Coordinates.csv')
data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
df.sort_values('PostalCode',ignore_index=True,inplace=True)

In [38]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Queen's Park,Ontario Provincial Government
1,M1C,East York,Woodbine Gardens
2,M1E,Etobicoke,Martin Grove
3,M1G,Scarborough,Port Union
4,M1H,Downtown Toronto,St. James Town


In [42]:
df_new = pd.merge(df,data,on = df['PostalCode']).drop(['key_0','Postal Code'],axis=1)
df_new.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Queen's Park,Ontario Provincial Government,43.806686,-79.194353
1,M1C,East York,Woodbine Gardens,43.784535,-79.160497
2,M1E,Etobicoke,Martin Grove,43.763573,-79.188711
3,M1G,Scarborough,Port Union,43.770992,-79.216917
4,M1H,Downtown Toronto,St. James Town,43.773136,-79.239476
5,M1J,Scarborough,Guildwood,43.744734,-79.239476
6,M1K,Scarborough,Woburn,43.727929,-79.262029
7,M1L,North York,Bathurst Manor,43.711112,-79.284577
8,M1M,West Toronto,Dufferin,43.716316,-79.239476
9,M1N,North York,York University,43.692657,-79.264848


In [48]:
"""import geocoder # import geocoder
lats, lons = [], []
count = 0
for postal_code in df1['PostalCode'].values:
    lat_lng_coords = None
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        lats.append(lat_lng_coords[0])
        lons.append(lat_lng_coords[1])"""

In [ ]:
# Part 3
'-----------------------------------------------------------------------------'

### Explore and cluster the neighborhoods in Toronto

In [55]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


In [58]:

import folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [61]:
north_york_toronto = df_new[df_new['Borough'] == 'North York'].reset_index(drop=True)

north_york_toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1L,North York,Bathurst Manor,43.711112,-79.284577
1,M1N,North York,York University,43.692657,-79.264848
2,M1T,North York,Downsview West,43.781638,-79.304302
3,M1W,North York,Humber Summit,43.799525,-79.318389
4,M1X,North York,Newtonbrook,43.836125,-79.205636


In [62]:
address = 'North York ,Toronto, Ontario'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude


map_dwontown = folium.Map(location=[latitude, longitude], zoom_start= 11)


for lat, lng, borough, neighborhood in zip(north_york_toronto['Latitude'], north_york_toronto['Longitude'], 
                                           north_york_toronto['Borough'], north_york_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dwontown)  
    
map_dwontown

In [64]:
latitude = north_york_toronto.loc[0, 'Latitude'] # neighborhood latitude value
longitude = north_york_toronto.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = north_york_toronto.loc[0, 'Neighborhood'] # neighborhood name

CLIENT_ID = '********' # your Foursquare ID
CLIENT_SECRET = '********' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

LIMIT = 100
radius =1000
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, latitude,longitude, radius, LIMIT)

# gettig the venues data form Forsquare API in json format
results = requests.get(url).json()


In [66]:
from pandas.io.json import json_normalize
venues = results['response']['groups'][0]['items']
venues_df = json_normalize(venues) 

<ipython-input-66-38763e2bcde8>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues_df = json_normalize(venues)


In [67]:
cols = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
venues_df = venues_df.loc[:, cols]
pd.set_option("display.max_colwidth" , -1)
venues_df.head()

<ipython-input-67-2935e484cc52>:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth" , -1)


,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Gingerman Restaurant,"[{'id': '4bf58dd8d48988d1c4941735', 'name': 'Restaurant', 'pluralName': 'Restaurants', 'shortName': 'Restaurant', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_', 'suffix': '.png'}, 'primary': True}]",43.707844,-79.295580
1,Dairy Queen,"[{'id': '4bf58dd8d48988d1c9941735', 'name': 'Ice Cream Shop', 'pluralName': 'Ice Cream Shops', 'shortName': 'Ice Cream', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/icecream_', 'suffix': '.png'}, 'primary': True}]",43.710378,-79.290701
2,Mexitaco,"[{'id': '4bf58dd8d48988d1c1941735', 'name': 'Mexican Restaurant', 'pluralName': 'Mexican Restaurants', 'shortName': 'Mexican', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/mexican_', 'suffix': '.png'}, 'primary': True}]",43.708595,-79.295700
3,TD Canada Trust,"[{'id': '4bf58dd8d48988d10a951735', 'name': 'Bank', 'pluralName': 'Banks', 'shortName': 'Bank', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/financial_', 'suffix': '.png'}, 'primary': True}]",43.705519,-79.278012
4,Starbucks,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'Coffee Shop', 'pluralName': 'Coffee Shops', 'shortName': 'Coffee Shop', 'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/coffeeshop_', 'suffix': '.png'}, 'primary': True}]",43.703634,-79.277661


In [68]:
pd.reset_option('display.max_colwidth')
venues_df['venue.categories'] = venues_df.apply(lambda x: x['venue.categories'][0]['name'], axis=1)
venues_df.head()

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Gingerman Restaurant,Restaurant,43.707844,-79.295580
1,Dairy Queen,Ice Cream Shop,43.710378,-79.290701
2,Mexitaco,Mexican Restaurant,43.708595,-79.295700
3,TD Canada Trust,Bank,43.705519,-79.278012
4,Starbucks,Coffee Shop,43.703634,-79.277661


In [69]:
venues_df.columns = [col.split(".")[-1] for col in venues_df.columns]
venues_df.head()

,name,categories,lat,lng
0,Gingerman Restaurant,Restaurant,43.707844,-79.295580
1,Dairy Queen,Ice Cream Shop,43.710378,-79.290701
2,Mexitaco,Mexican Restaurant,43.708595,-79.295700
3,TD Canada Trust,Bank,43.705519,-79.278012
4,Starbucks,Coffee Shop,43.703634,-79.277661


In [73]:
def get_near_by_venues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(name, lat, lng, 
                             v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'],
                             v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue in venues_list for item in venue])
    nearby_venues.columns = ['Neighborhood','Neighborhood Latitude', 'Neighborhood Longitude', 
                             'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return nearby_venues

In [74]:
north_york_toronto_venues = get_near_by_venues(names=north_york_toronto['Neighborhood'],latitudes=north_york_toronto['Latitude'],
                                   longitudes=north_york_toronto['Longitude'])

In [75]:
north_york_toronto_onehot = pd.get_dummies(north_york_toronto_venues[['Venue Category']], prefix= "", prefix_sep= " ")

north_york_toronto_onehot['Neighborhood'] = north_york_toronto_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [north_york_toronto_onehot.columns[-1]] + list(north_york_toronto_onehot.columns[:-1])
north_york_toronto_onehot = north_york_toronto_onehot[fixed_columns]

north_york_toronto_onehot.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,Bathurst Manor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Bathurst Manor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Bathurst Manor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Bathurst Manor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Bathurst Manor,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [76]:
north_york_toronto_grouped = north_york_toronto_onehot.groupby('Neighborhood').mean().reset_index()

north_york_toronto_grouped.head()

,Neighborhood,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thai Restaurant,Theater,Theme Restaurant,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Wine Bar,Yoga Studio
0,Downsview North,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000
1,Henry Farm,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.038462,0.0,...,0.038462,0.000000,0.000000,0.038462,0.0,0.00,0.0,0.0,0.0,0.000000
2,Lawrence Manor East,0.013158,0.0,0.0,0.0,0.0,0.0,0.0,0.013158,0.0,...,0.013158,0.013158,0.013158,0.000000,0.0,0.00,0.0,0.0,0.0,0.026316
3,Maple Leaf Park,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.090909,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0,0.000000
4,Oriole,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.25,0.0,0.0,0.0,0.000000


In [77]:
num_top_venues = 5

for hood in north_york_toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = north_york_toronto_grouped[north_york_toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    
   
    temp = temp.iloc[1:]
    temp['freq'] = round(temp['freq'].astype(float),2)# converting into float type and # taking round values
    temp = temp.sort_values('freq', ascending=False).reset_index(drop=True) # sorting the dataframe by 'freq' in decreasing order
    print(temp.head(num_top_venues))
    print('\n')

---- Downsview North----
             venue  freq
0    Grocery Store  0.25
1             Park  0.25
2    Shopping Mall  0.25
3             Bank  0.25
4   Adult Boutique  0.00


---- Henry Farm----
                 venue  freq
0           Restaurant  0.08
1   Italian Restaurant  0.08
2          Coffee Shop  0.08
3       Sandwich Place  0.08
4     Greek Restaurant  0.04


---- Lawrence Manor East----
                  venue  freq
0           Coffee Shop  0.08
1   Japanese Restaurant  0.07
2      Sushi Restaurant  0.07
3            Restaurant  0.04
4               Gay Bar  0.04


---- Maple Leaf Park----
          venue  freq
0           Gym  0.09
1        Bakery  0.09
2   Pizza Place  0.09
3    Restaurant  0.09
4          Café  0.09


---- Oriole----
                 venue  freq
0   Turkish Restaurant  0.25
1       Sandwich Place  0.25
2                  Bar  0.25
3       Discount Store  0.25
4       Adult Boutique  0.00


---- Silver Hills----
                 venue  freq
0             

In [78]:
def common_venues(row, num_top_venues):
    row = row.iloc[1:]
    row_sorted = row.sort_values(ascending=False)
    
    return row_sorted.index.values[0:num_top_venues]

In [79]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Neighborhood'] = north_york_toronto_grouped['Neighborhood']

for ind in np.arange(north_york_toronto_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = common_venues(north_york_toronto_grouped.iloc[ind, :], num_top_venues)

venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downsview North,Park,Grocery Store,Bank,Shopping Mall,Comfort Food Restaurant,Dessert Shop,Donut Shop,Dog Run,Dive Bar,Distribution Center
1,Henry Farm,Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Fast Food Restaurant,Cosmetics Shop,Cupcake Shop,Pharmacy,Pizza Place,Café
2,Lawrence Manor East,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Hotel,Men's Store,Mediterranean Restaurant,Pub
3,Maple Leaf Park,American Restaurant,Pizza Place,Bakery,Seafood Restaurant,Mexican Restaurant,Restaurant,Fast Food Restaurant,Café,Gym,Pharmacy
4,Oriole,Turkish Restaurant,Sandwich Place,Bar,Discount Store,Yoga Studio,Deli / Bodega,Dog Run,Dive Bar,Distribution Center,Diner


In [ ]:
# Clustering

In [80]:
from sklearn.cluster import KMeans
k = 5

X = north_york_toronto_grouped.drop('Neighborhood', axis = 1)

# run k-means clustering
kmeans = KMeans(n_clusters = k, random_state=0)
kmeans.fit(X)

KMeans(n_clusters=5, random_state=0)

In [81]:
venues_sorted['Cluster_Labels']=  kmeans.labels_

north_york_toronto_merged = north_york_toronto
# merge top venues_sorted with toronto_data
north_york_toronto_merged = north_york_toronto_merged.join(venues_sorted.set_index('Neighborhood'), on='Neighborhood')

north_york_toronto_merged.head() 

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
0,M1L,North York,Bathurst Manor,43.711112,-79.284577,Bus Line,Bakery,Metro Station,Ice Cream Shop,Park,Soccer Field,Intersection,Dive Bar,Distribution Center,Discount Store,1.0
1,M1N,North York,York University,43.692657,-79.264848,College Stadium,General Entertainment,Café,Skating Rink,Department Store,Donut Shop,Dog Run,Dive Bar,Distribution Center,Discount Store,1.0
2,M1T,North York,Downsview West,43.781638,-79.304302,Pizza Place,Fast Food Restaurant,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Convenience Store,Intersection,Bank,Pharmacy,Fried Chicken Joint,1.0
3,M1W,North York,Humber Summit,43.799525,-79.318389,Coffee Shop,Furniture / Home Store,Sandwich Place,Fast Food Restaurant,Bank,Chinese Restaurant,Cosmetics Shop,Breakfast Spot,Pharmacy,Pizza Place,1.0
4,M1X,North York,Newtonbrook,43.836125,-79.205636,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
from matplotlib import cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k)
ys = [i+x+(i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(north_york_toronto_merged['Latitude'], north_york_toronto_merged['Longitude'], north_york_toronto_merged['Neighborhood'], north_york_toronto_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [97]:
# Cluster 1 
north_york_toronto_merged.loc[north_york_toronto_merged['Cluster_Labels'] == 0, north_york_toronto_merged.columns[[1] + list(range(5, north_york_toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
6,North York,Park,Food & Drink Shop,Yoga Studio,Department Store,Donut Shop,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner,0.0


In [98]:
# Cluster 2
north_york_toronto_merged.loc[north_york_toronto_merged['Cluster_Labels'] == 1, north_york_toronto_merged.columns[[1] + list(range(5, north_york_toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
0,North York,Bus Line,Bakery,Metro Station,Ice Cream Shop,Park,Soccer Field,Intersection,Dive Bar,Distribution Center,Discount Store,1.0
1,North York,College Stadium,General Entertainment,Café,Skating Rink,Department Store,Donut Shop,Dog Run,Dive Bar,Distribution Center,Discount Store,1.0
2,North York,Pizza Place,Fast Food Restaurant,Thai Restaurant,Italian Restaurant,Chinese Restaurant,Convenience Store,Intersection,Bank,Pharmacy,Fried Chicken Joint,1.0
3,North York,Coffee Shop,Furniture / Home Store,Sandwich Place,Fast Food Restaurant,Bank,Chinese Restaurant,Cosmetics Shop,Breakfast Spot,Pharmacy,Pizza Place,1.0
7,North York,Park,Grocery Store,Bank,Shopping Mall,Comfort Food Restaurant,Dessert Shop,Donut Shop,Dog Run,Dive Bar,Distribution Center,1.0
8,North York,Coffee Shop,Pizza Place,Portuguese Restaurant,Hockey Arena,Intersection,Yoga Studio,Deli / Bodega,Dive Bar,Distribution Center,Discount Store,1.0
9,North York,Coffee Shop,Gastropub,Brewery,Café,Bakery,American Restaurant,Convenience Store,Cheese Shop,Clothing Store,Pet Store,1.0
10,North York,Coffee Shop,Bakery,Café,Restaurant,Italian Restaurant,Pizza Place,Pub,Playground,Diner,Indian Restaurant,1.0
11,North York,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Yoga Studio,Hotel,Men's Store,Mediterranean Restaurant,Pub,1.0
12,North York,Coffee Shop,Café,Hotel,Cocktail Bar,Cosmetics Shop,Restaurant,Park,Clothing Store,Moroccan Restaurant,Seafood Restaurant,1.0


In [99]:
# Cluster 3
north_york_toronto_merged.loc[north_york_toronto_merged['Cluster_Labels'] == 2, north_york_toronto_merged.columns[[1] + list(range(5, north_york_toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
29,North York,Rental Car Location,Garden Center,Drugstore,Dance Studio,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner,Dessert Shop,2.0


In [100]:
# Cluster 4
north_york_toronto_merged.loc[north_york_toronto_merged['Cluster_Labels'] == 3, north_york_toronto_merged.columns[[1] + list(range(5, north_york_toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
27,North York,Baseball Field,Yoga Studio,Escape Room,Drugstore,Donut Shop,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner,3.0


In [101]:
# Cluster 5
north_york_toronto_merged.loc[north_york_toronto_merged['Cluster_Labels'] == 4, north_york_toronto_merged.columns[[1] + list(range(5, north_york_toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster_Labels
25,North York,River,Yoga Studio,College Auditorium,Donut Shop,Dog Run,Dive Bar,Distribution Center,Discount Store,Diner,Dessert Shop,4.0
